In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

# Quick recap: How to use Dates & Times with pandas

## Basic building block: `pd.Timestamp`

In [ ]:
time_stamp = pd.Timestamp(datetime(2022, 12, 12))
time_stamp

Attributes to store time-specific information

In [ ]:
time_stamp.year

In [ ]:
time_stamp.day_name()

## More building blocks: `pd.Period` and `freq`

In [ ]:
period = pd.Period("2017-01")
period

Period object has freq attribute to store frequency info

In [ ]:
# convert to daily
period.asfreq("D")

Convert Period to Timestamp and back

In [ ]:
period.to_timestamp()

In [ ]:
period.to_timestamp().to_period("M")

Frequency info enables basic arithmetic

In [ ]:
period + 2

## Sequences of Dates & Times

In [ ]:
index = pd.date_range(start="2022-1-1", periods=12, freq="M")
index

`DateTimeIndex`: sequency of `Timestamp` objects with frequency info

In [ ]:
index[0]

In [ ]:
# index as monthly period to plot monthly aggregates
index.to_period()

# Indexing & resampling time series

## Let's load some data

In [ ]:
ts = pd.read_csv("AAPL.csv", index_col="Date",  parse_dates=True)
ts.info()

In [ ]:
ts.head()

In [ ]:
ts.plot(y="Close", title="Apple stock closing price");

## Splitting TS by date

Select all data for 2018

In [ ]:
# Approach 1
ts[ts.index.year == 2018]

In [ ]:
# Approch 2, less verbose
ts["2018"]

Selecting only data from summer months of 2018

In [ ]:
# the verbose way...
ts[(ts.index >= "2018-6") & (ts.index <= "2018-9")]

In [ ]:
# ... the concise way. Slicing includes last month unlike Python slicing
ts["2018-6" : "2018-8"]

In [ ]:
# This also works
ts["2018-6":]

## Changing frequency

In [ ]:
ts.head()

In [ ]:
ts.index

**Upsampling** - higher frequency implies new dates -> missing data

In [ ]:
# Set daily frequency
ts_daily = ts.asfreq("D").head()
ts_daily

In [ ]:
ts_daily.index

Options to impute missing data: mean,media, last value

In [ ]:
ts.asfreq("D").ffill().head()

Why changing TS frequency relevant? For example, to merged time series with different frequencies

# Merging time series

Client case example (data made availaby by courtesy of Daan Salome) 

In [ ]:
# pressurization time
pt = pd.read_csv("PressurisationSeconds.csv", index_col="_time", parse_dates=True).sort_index()
pt.head()

In [ ]:
pt.info()

## Side note on Time Zones

The "+00:00" at the end of the timestamp indicate time zone information. We know the time zone is UTC

In [ ]:
pt.index.tz

To convert a time zone aware pandas object from one time zone to another, you can use the `tz_convert` method.

In [ ]:
pt.index[:4]

In [ ]:
pt.index.tz_convert("US/Eastern")[:4]

In [ ]:
pt.index.tz_convert("US/Pacific")[:4]

In [ ]:
# convert DF to different TZ
pt.tz_convert("US/Pacific").head()

How to strip the TZ information if it annoys us

In [ ]:
pt = pt.tz_localize(None)
pt.head()

## Load 2nd Time series

In [ ]:
loads = pd.read_csv("smBpTestStandLoadTest.State.csv", index_col="_time", parse_dates=True).sort_index()
loads = loads.tz_localize(None)
loads.head(10)

## Solving for different frequencies
How to _align_ the data in second TS on the data in the first TS

**Reindexing**  is useful in preparation for combining two time series data sets. 

If data is unavailable for one of the new index dates or times, you must tell pandas how to fill it
in. Otherwise, pandas will fill with NaN by default.

In [ ]:
pt.reindex(loads.index)

In [ ]:
pt_reindex = pt.reindex(loads.index, method="pad")
pt_reindex

In [ ]:
# now we can join datasets
joined = loads.join(pt_reindex)
joined

## `merge_asof`

**Could be viewed as reindex and merge in one operation**

Similar to a left-join except that we match on nearest key rather than equal keys.  For each row in the left DataFrame, we select the last row in the right DataFrame whose on key is less than the left’s key. 

In [ ]:
loads.head()

In [ ]:
pt.head()

In [ ]:
pd.merge_asof(loads, pt, left_index=True, right_index=True).head()

# Time series analysis: seasonality, trend and noise

CO2 levels in Mauai Hawaii

In [ ]:
co2_levels = pd.read_csv("ch2_co2_levels.csv", index_col="datestamp", parse_dates=True).dropna()
co2_levels

In [ ]:
co2_levels.index

In [ ]:
co2_levels.plot(title="CO2 levels Mauna Loa observatory, Hawaii");

## Autocorrelations

Let's manually calculate how auto-correlated our signal is

In [ ]:
# we shift (i.e. delay)  the data in multiples of 1 week
shifts = range(1, 8)

for ii in shifts:
    co2_levels[f"lag_{ii}"] = co2_levels["co2"].shift(ii)

co2_levels.head(10)

In [ ]:
co2_levels.plot()

In [ ]:
co2_levels.corr()

... and now automatically 

In [ ]:
from statsmodels.graphics import tsaplots
tsaplots.plot_acf(co2_levels["co2"], lags=60);

**Interpret autocorrelation plots**

If autocorrelation values are close to O, then values between consecutive observations are not correlated with one another.
Inversely, autocorrelations values close to 1 or -1 indicate that there exists strong positive or negative correlations between
consecutive observations, respectively.

In order to help you asses how trustworthy these autocorrelation values are, the plot-acf() function also returns
confidence intervals (represented as blue shaded regions). If an autocorrelation value goes beyond the confidence interval
region, you can assume that the observed autocorrelation value is statistically significant.

## Time series decomposition

In [ ]:
import statsmodels.api as sm

`statsmodels` is a Python module that provides classes and functions for the estimation of many different statistical models, as well as for conducting statistical tests, and statistical data
exploration.

In [ ]:
decomposition = sm.tsa.seasonal_decompose(co2_levels["co2"])          

How to fix the error we've just received

In [ ]:
co2_levels.index

In [ ]:
# built-in pandas attribute not really helpful...
co2_levels.index.inferred_freq

In [ ]:
freqs = co2_levels.index[1:] - co2_levels.index[:-1]
freqs

In [ ]:
freqs.value_counts()

In [ ]:
co2_levels_7d  = co2_levels.asfreq("7D").ffill()

In [ ]:
co2_levels_7d.index

In [ ]:
# Let's try again to decompose the TS
decomposition = sm.tsa.seasonal_decompose(co2_levels_7d["co2"])

# Temporarily changing plotting params 
with plt.rc_context({"figure.figsize": (11, 9)}):
    decomposition.plot()                                      

The following additive model is fit to the data

Y[t] = T[t] + S[t] + e[t]

The results are obtained by first estimating the trend by applying a convolution filter to the data. The trend is then removed
from the series and the average of this de-trended series for each period is the returned seasonal component.

### Extracting components from TS decomposition

In [ ]:
print(decomposition.seasonal)

In [ ]:
seas = decomposition.seasonal
fig, ax = plt.subplots(figsize=(11,3))
seas.plot(ax=ax)
ax.set(xlabel="Date",
       title="Seasonal values of time series");

In [ ]:
trend = decomposition.trend
fig, ax = plt.subplots(figsize=(11,3))
trend.plot(ax=ax)
ax.set(xlabel="Date",
       title="Trend values of time series");

However, the analysis does not **automatically** provide an answer to the question:   **What is the seasonality of my data?**

Eyeballing the seasonal pattern, we can visually infer that the time series has annual seasonality with peaks in the month of May/June

_Could we think of an automatic approach to infer the seasonality period?_